# インポート

In [1]:
import preprocessing
%load_ext autoreload

# データ加工

In [2]:
# レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [3]:
# 重複チェック
results_preprocessed.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [4]:
# 欠損チェック
results_preprocessed.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
rank           0
wakuban        0
umaban         0
sex            0
age            0
weight         0
weight_diff    0
tansho_odds    0
popularity     0
impost         0
dtype: int64

In [2]:
# 馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()

In [3]:
horse_results_preprocessed

,horse_id,date,rank,price,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
0,2018102264,2024-01-21,10.0,0.0,1.0,1.0,0,1200,1.0,4.0,16
1,2018102264,2023-12-09,5.0,184.0,0.5,0.0,0,1200,0.0,4.0,16
2,2018102264,2023-07-23,4.0,275.0,0.2,0.0,0,1150,0.0,5.0,11
3,2018102264,2023-06-11,7.0,0.0,0.3,1.0,0,1200,1.0,4.0,16
4,2018102264,2023-04-02,9.0,0.0,1.5,0.0,0,1200,0.0,4.0,16
...,...,...,...,...,...,...,...,...,...,...,...
161380,2020102750,2023-12-27,1.0,40.0,0.0,0.0,0,1400,0.0,NaN,7
161381,2020102750,2023-12-06,3.0,8.0,0.4,0.0,0,1400,0.0,NaN,8
161382,2020102750,2023-11-07,5.0,2.0,2.3,1.0,0,800,1.0,NaN,7
161383,2020102750,2023-09-03,14.0,0.0,3.4,0.0,0,1700,0.0,1.0,16


In [4]:
horse_results_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159387 entries, 0 to 161384
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   horse_id      159387 non-null  int64         
 1   date_horse    159387 non-null  datetime64[ns]
 2   rank          159387 non-null  float64       
 3   price         159387 non-null  float64       
 4   rank_diff     159131 non-null  float64       
 5   weather       159174 non-null  float64       
 6   race_type     159387 non-null  int64         
 7   course_len    159387 non-null  int64         
 8   ground_state  159384 non-null  float64       
 9   race_class    98409 non-null   float64       
 10  n_horses      159387 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(4)
memory usage: 14.6 MB


In [2]:
# レース情報テーブルの加工
race_info = preprocessing.process_race_info()

In [3]:
race_info

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202301010101,2023-07-22,1,0.0,1200,0,0,1,1
1,202301010102,2023-07-22,0,0.0,1000,0,0,1,1
2,202301010103,2023-07-22,0,0.0,1700,0,0,1,1
3,202301010104,2023-07-22,1,0.0,1500,0,0,1,1
4,202301010105,2023-07-22,0,0.0,1700,0,0,0,1
...,...,...,...,...,...,...,...,...,...
3451,202310030808,2023-09-03,0,0.0,1000,0,0,2,10
3452,202310030809,2023-09-03,1,0.0,1200,0,0,3,10
3453,202310030810,2023-09-03,0,0.0,1700,0,0,4,10
3454,202310030811,2023-09-03,1,0.0,1200,0,0,5,10


In [4]:
race_info.isnull().sum()

race_id           0
date              0
race_type         0
around          127
course_len       27
weather           0
ground_state      0
race_class        0
place             0
dtype: int64

In [4]:
from feature_engineering import FeatureCreator

In [5]:
# インスタンス化
fc = FeatureCreator()
fc.results

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,wakuban,umaban,sex,age,weight,weight_diff,tansho_odds,popularity,impost
0,202301010101,2021100648,1095,1128,281008,5,1,1,1,2,434,-10,23.9,5,55.0
1,202301010101,2021100159,1157,1186,170800,6,2,2,0,2,454,-6,61.8,7,55.0
2,202301010101,2021100265,1197,1192,320803,7,3,3,1,2,404,-2,18.8,4,53.0
3,202301010101,2021105553,5339,1160,680031,4,4,4,1,2,450,2,16.6,3,55.0
4,202301010101,2021101429,1170,1026,425031,1,5,5,0,2,452,-4,1.2,1,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,1208,1161,17803,1,6,8,1,4,442,-2,3.5,1,53.0
47269,202310030812,2020103754,1116,1104,651031,6,7,9,0,3,468,0,15.2,8,55.0
47270,202310030812,2020100400,1193,1092,180800,4,7,10,1,3,458,6,23.1,9,50.0
47271,202310030812,2020103663,732,1092,226800,2,8,11,2,3,486,-6,5.0,3,55.0


In [6]:
# 特徴量作成の実行
features = fc.create_features()

In [7]:
features

,race_id,horse_id,date,jockey_id,trainer_id,owner_id,rank,wakuban,umaban,sex,...,race_class,place,rank_3races,price_3races,rank_5races,price_5races,rank_10races,price_10races,rank_1000races,price_1000races
0,202301010101,2021100648,2023-07-22,1095,1128,281008,5,1,1,1,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,202301010101,2021100159,2023-07-22,1157,1186,170800,6,2,2,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202301010101,2021100265,2023-07-22,1197,1192,320803,7,3,3,1,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202301010101,2021105553,2023-07-22,5339,1160,680031,4,4,4,1,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,202301010101,2021101429,2023-07-22,1170,1026,425031,1,5,5,0,...,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,2023-09-03,1208,1161,17803,1,6,8,1,...,2,10,5.666667,46.666667,7.4,28.0,8.3,22.3,6.321101,45.001835
47269,202310030812,2020103754,2023-09-03,1116,1104,651031,6,7,9,0,...,2,10,5.666667,46.666667,7.4,28.0,8.3,22.3,6.321101,45.001835
47270,202310030812,2020100400,2023-09-03,1193,1092,180800,4,7,10,1,...,2,10,5.666667,46.666667,7.4,28.0,8.3,22.3,6.321101,45.001835
47271,202310030812,2020103663,2023-09-03,732,1092,226800,2,8,11,2,...,2,10,5.666667,46.666667,7.4,28.0,8.3,22.3,6.321101,45.001835


In [8]:
features.isnull().sum()

race_id                0
horse_id               0
date                   0
jockey_id              0
trainer_id             0
owner_id               0
rank                   0
wakuban                0
umaban                 0
sex                    0
age                    0
weight                 0
weight_diff            0
tansho_odds            0
popularity             0
impost                 0
race_type              0
around              1371
course_len           317
weather                0
ground_state           0
race_class             0
place                  0
rank_3races        10230
price_3races       10230
rank_5races        10230
price_5races       10230
rank_10races       10230
price_10races      10230
rank_1000races     10230
price_1000races    10230
dtype: int64